# INST326 — Week 5 Exercises: Encapsulation & Data Hiding (Library Management Project)

**Scope guard:** These exercises stay within *Week 5 topics only* — encapsulation, private/protected attributes, getters/setters, and `@property` with validation. **Avoid** features introduced in later weeks (e.g., class/staticmethods, inheritance, abstract classes, advanced error handling/test frameworks).


## Python skills you’ll need (Week 5 scope)

- Defining classes and creating instances (`class`, `__init__`, `self`)
- Naming conventions for attribute privacy (`_protected`, `__private`/name mangling)
- Encapsulation patterns:
  - Getter/setter methods
  - `@property` / `@<name>.setter`
  - Simple validation logic inside setters (e.g., type/empty checks, ranges)
- Basic built-in types and operations (`str`, `int`, `float`, `bool`, `len`, `in`)
- F-strings for messages (optional)
- Simple list/dict usage for holding objects (no advanced collections needed)
- Basic control flow (`if`, `for`)
- (Optional but allowed) raising simple exceptions like `ValueError` in setters for invalid data (no try/except blocks required here)


---
## How to use this notebook

Each exercise includes a short prompt and a starter code cell. Keep your solutions within **Week 5** boundaries. If an exercise references existing domain classes, you may stub minimal versions as needed inside the same cell (or reuse earlier cells). Keep code readable and well-encapsulated.


### Exercise 1 — Encapsulated `Book` identifiers
Create a `Book` class with *private* attributes `__isbn` (string) and `__title` (string). Expose read-only access via `@property` for both. Prevent empty strings at construction time using validation inside `__init__`. If invalid, either normalize to a sensible default or raise `ValueError` (your choice).

In [2]:
class Book:
    def __init__(self, isbn: str, title: str):
        self.__isbn = isbn
        self.__title = title


    @property
    def isbn(self) -> str:
        return self.__isbn

    @property
    def title(self) -> str:
        return self.__title

# Demo
b = Book("9780132350884", "Clean Code")
print(b.isbn, b.title)


9780132350884 Clean Code


### Exercise 2 — Writable `Member` email with validation
Define a `Member` class with private `__email`. Provide `email` property and setter that requires `@` in the email and trims whitespace. Reject obviously invalid emails by raising `ValueError`. Provide a `__repr__` that shows member id and email.

In [22]:
class Member:
    def __init__(self, member_id: int, email: str):
        self.__email = None
        self.email = email
        self.member_id = member_id
    def __repr__(self) -> str:
        return f"Member(id= {self.member_id}, email= {self.__email})"

    @property
    def email(self) -> str:
        return self.__email
        
    @email.setter
    def email(self, value: str) -> None:
        value = str(value).strip()
        if "@" not in value :
            raise ValueError("Invalid email: must contain '@'")
        self.__email = value
        
# Demo
m = Member(101, "  user@example.org ")
print(m, m.email)


Member(id= 101, email= user@example.org) user@example.org


### Exercise 3 — Protected page count with non-negative constraint
Extend `Book` (or create a fresh class here) with a *protected* attribute `_pages` and a `pages` property. Setter must coerce to `int` and ensure it’s ≥ 1 (treat 0/negatives as invalid).

In [27]:
class PagedBook:
    def __init__(self, title: str, pages):
        self._pages = None
        self.pages = pages

    @property
    def pages(self) -> int:
        return self._pages

    @pages.setter
    def pages(self, value) -> None:
        value = int(value)
        if value <= 0:
            raise ValueError("Invalid page: must be greater than 0")
        self._pages = value
# Demo
pb = PagedBook("Fluent Python", 700)
print(pb.pages)


700


### Exercise 4 — Loan period property with range check
Create a `LoanPolicy` class with private `__max_days`. Provide property/settter ensuring 1 ≤ `max_days` ≤ 60. Default to 21 days.

In [30]:
class LoanPolicy:
    def __init__(self, max_days: int = 21):
        self.__max_days = None
        self.max_days = max_days

    @property
    def max_days(self) -> int:
        return self.__max_days

    @max_days.setter
    def max_days(self, value: int) -> None:
        if value <= 0 or value > 60:
            raise ValueError("Invalid max day count: Must be between 1 and 60")
        self.__max_days = value

# Demo
p = LoanPolicy()
print(p.max_days)


21


### Exercise 5 — Read-only `copy_id`
Create a `Copy` class representing a physical copy of a book with private `__copy_id` and protected `_condition` ('good', 'fair', 'poor'). Only `copy_id` is read-only. `condition` is a property with setter that restricts values to the allowed set.

In [53]:
class Copy:
    def __init__(self, copy_id: str, condition: str = "good"):
        self.__copy_id = copy_id
        self._condition = None
        self.condition = condition

    @property
    def copy_id(self) -> str:
        return self.__copy_id

    @property
    def condition(self) -> str:
        return self._condition

    @condition.setter
    def condition(self, value: str) -> None:
        if value != "fair" and value != "good" and value != "poor":
            raise ValueError("Condition type error: value outside of allowed set")
        self._condition = value
        

# Demo
c = Copy("C-1001", "fair")
print(c.copy_id, c.condition)


C-1001 fair


### Exercise 6 — Late fee rate with type safety
Create a `FeeSchedule` with private `__late_fee_per_day` (float). Property/settter must coerce numeric strings like '0.25' to float, and reject negatives.

In [54]:
class FeeSchedule:
    def __init__(self, late_fee_per_day):
        # TODO: set via property and validate non-negative float
        self.__late_fee_per_day = None
        self.late_fee_per_day = late_fee_per_day
        
    @property
    def late_fee_per_day(self) -> float:
        return self.__late_fee_per_day

    @late_fee_per_day.setter
    def late_fee_per_day(self, value) -> None:
        value = float(value)
        if value < 0:
            raise ValueError("Can not have negative fees")
        self.__late_fee_per_day = value

# Demo
fs = FeeSchedule("0.35")
print(fs.late_fee_per_day)


0.35


### Exercise 7 — Aggregating encapsulated objects (simple list)
Create a `Shelf` with a *protected* list `_copies`. Implement `add_copy(copy)` that accepts only `Copy` instances. Expose a read-only `copies` property returning a **tuple** (to prevent external mutation).

In [61]:
class Shelf:
    def __init__(self):
        self._copies = []

    def add_copy(self, copy) -> None:
        if not isinstance(copy, Copy):
            raise TypeError("Only Copy instances can be added to Shelf")
        self._copies.append(copy)

    @property
    def copies(self):
        return tuple(self._copies)

# Demo (you can reuse your Copy class from Exercise 5)
sh = Shelf()
sh.add_copy(Copy("C-1"))
sh.add_copy(Copy("c-1002"))
print(sh.copies)


(<__main__.Copy object at 0x108e82d80>, <__main__.Copy object at 0x108e82a50>)


### Exercise 8 — Borrower name normalization
Create a `Borrower` with private `__first_name` and `__last_name`. Properties should normalize spacing and capitalization (`title()` is fine). Provide a read-only property `full_name`.

In [66]:
class Borrower:
    def __init__(self, first_name: str, last_name: str):
        self.__first_name = None
        self.__last_name = None
        self.first_name = first_name
        self.last_name = last_name

    @property
    def first_name(self) -> str:
        return self.__first_name

    @first_name.setter
    def first_name(self, value: str) -> None:
        value = value.strip().capitalize()
        self.__first_name = value

    @property
    def last_name(self) -> str:
        return self.__last_name

    @last_name.setter
    def last_name(self, value: str) -> None:
        value = value.strip().capitalize()
        self.__last_name = value

    @property
    def full_name(self) -> str:
        return f"{self.first_name} {self.last_name}"

# Demo
b = Borrower("  aLiCe ", "  johnson ")
print(b.full_name)  # -> 'Alice Johnson'


Alice Johnson


### Exercise 9 — Minimum age policy for membership
Create `MembershipPolicy` with private `__min_age` default 13. Property/settter must enforce 0 ≤ min_age ≤ 120 and coerce to `int`. Add method `is_eligible(age)` that checks age against the policy (no class/staticmethods).

In [68]:
class MembershipPolicy:
    def __init__(self, min_age: int = 13):
        self.min_age = min_age

    @property
    def min_age(self) -> int:
        return self.__min_age

    @min_age.setter
    def min_age(self, value) -> None:
        value = int(value)
        if not (0 <= value <= 120):
            raise ValueError("min_age must be between 0 and 120")
        self.__min_age = value

    def is_eligible(self, age) -> bool:
        age = int(age)
        return age >= self.min_age

# Demo
mp = MembershipPolicy()
print(mp.is_eligible(12), mp.is_eligible(15))


False True


### Exercise 10 — Encapsulated `Loan` dates (strings allowed)
Create `Loan` with private `__start_date` and `__due_date` stored as ISO date strings 'YYYY-MM-DD'. Properties should validate format *lightly* via length and digit checks (no datetime imports).

In [73]:
class Loan:
    def __init__(self, start_date: str, due_date: str):
        self.start_date = start_date
        self.due_date = due_date

    @property
    def start_date(self) -> str:
        return self.__start_date

    @start_date.setter
    def start_date(self, value: str) -> None:
        if not isinstance(value, str):
            raise ValueError("Value must be a string")
        if len(value) != 10 or value[4] != '-' or value[7] != '-':
            raise ValueError("Invalid start_date format: must be 'YYYY-MM-DD'")
        self.__start_date = value

    @property
    def due_date(self) -> str:
        return self.__due_date

    @due_date.setter
    def due_date(self, value: str) -> None:
        if not isinstance(value, str):
            raise ValueError("Value must be a string")
        if len(value) != 10 or value[4] != '-' or value[7] != '-':
            raise ValueError("Invalid start_date format: must be 'YYYY-MM-DD'")
        self.__due_date = value

# Demo
ln = Loan("2025-10-06", "2025-10-27")
print(ln.start_date, ln.due_date)


2025-10-06 2025-10-27


### Exercise 11 — Computed read-only property: `is_overdue`
On `Loan` (or your own class), add a read-only boolean property `is_overdue` that compares today-as-string provided to a method e.g., `mark_today(date_str)` that stores a private `__today`. No datetime libs; simple string comparison is acceptable.

In [76]:
class SimpleLoan:
    def __init__(self, due_date: str):
        self.__due_date = due_date
        self.__today = None

    def mark_today(self, date_str: str) -> None:
        if isinstance(date_str, str) and len(date_str) == 10 and date_str.count('-') == 2:
            self.__today = date_str
        else:
            raise ValueError("Invalid date format. Use 'YYYY-MM-DD'.")
    @property
    def is_overdue(self) -> bool:
        if self.__today is None:
            return False
        return self.__today > self.__due_date

# Demo
sl = SimpleLoan("2025-10-20")
sl.mark_today("2025-10-21")
print(sl.is_overdue)  # True


True


### Exercise 12 — Immutable `LibraryCard` number
Create `LibraryCard` with private `__number` (string). Provide only a read-only `number` property. Add optional `pin` with setter that requires 4 digits. Do **not** allow number changes after construction.

In [89]:
class LibraryCard:
    def __init__(self, number: str, pin: str | None = None):
        # TODO: number is immutable; pin uses property with 4-digit rule
        self.__number = number
        self.pin = None

    @property
    def number(self) -> str:
        return self.__number

    @property
    def pin(self) -> str | None:
        return self._pin

    @pin.setter
    def pin(self, value: str) -> None:
        value = str(value)
        if len(value) > 4 or len(value) < 4:
            raise ValueError("pin must be 4 digits")
        self._pin = value

# Demo
lc = LibraryCard("CARD-0001")
lc.pin = "1234"
print(lc.number, lc.pin)


CARD-0001 1234


### Exercise 13 — Title case normalization with `@property`
Add a `title` property to `Book`-like class that always returns title-cased text and stores trimmed private `__title`. Prevent setting titles shorter than 2 characters.

In [90]:
class TitledBook:
    def __init__(self, title: str):
        self.title = title

    @property
    def title(self) -> str:
        return self.__title

    @title.setter
    def title(self, value: str) -> None:
        normalized = value.strip()
        if len(normalized) < 2:
            raise ValueError("Title must be at least 2 characters long.")
        self.__title = normalized.title()

# Demo
tb = TitledBook("  the pragmatic programmer ")
print(tb.title)  # 'The Pragmatic Programmer'


The Pragmatic Programmer


### Exercise 14 — Encapsulated search query history
Create `SearchSession` with *protected* list `_queries`. Provide `add_query(q: str)` that stores trimmed non-empty strings only. Expose a read-only `queries` tuple property.

In [92]:
class SearchSession:
    def __init__(self):
        self._queries = []

    def add_query(self, q: str) -> None:
        trimmed = q.strip()
        if trimmed:
            self._queries.append(trimmed)

    @property
    def queries(self):
       return tuple(self._queries)

# Demo
ss = SearchSession()
ss.add_query("  design patterns ")
print(ss.queries)


('design patterns',)


### Exercise 15 — Simple `Tag` with sanitized names
Create `Tag` with private `__name`. Setter should lowercase, strip spaces, and collapse internal whitespace to single spaces. Reject tags longer than 30 characters.

In [4]:
class Tag:
    def __init__(self, name: str):
        self.name = name

    @property
    def name(self) -> str:
        return self.__name

    @name.setter
    def name(self, value: str) -> None:
        sanitized = value.strip().lower()
        parts = sanitized.split()
        sanitized = " ".join(parts)
        if len(sanitized) > 30:
            raise ValueError("Tag name must be 30 characters or fewer.")

        self.__name = sanitized

# Demo
t = Tag("  Object   Oriented   Design   ")
print(t.name)  # 'object oriented design'


object oriented design


### Exercise 16 — `BookCopy` status transitions via setter
Create `BookCopy` with private `__status` in {'available','on_loan','missing'}. The `status` setter should only allow transitions:
available → on_loan → available, and *any* → missing. Invalid transitions should be rejected (e.g., raise `ValueError`).

In [7]:
class BookCopy:
    def __init__(self, copy_id: str, status: str = "available"):
        self.copy_id = copy_id
        self.__status = None
        self.status = status

    @property
    def status(self) -> str:
        return self.__status

    @status.setter
    def status(self, value: str) -> None:
        allowed_statuses = {"available", "on_loan", "missing"}
        if value not in allowed_statuses:
            raise ValueError(f"Invalid status: {value}. Must be one of {allowed_statuses}")

        if value == "missing":
            self.__status = value
            return

        if self.__status is None:
            if value != "available":
                raise ValueError("Initial status must be 'available'")
            self.__status = value
            return

        valid_transitions = {
            "available": {"on_loan", "missing"},
            "on_loan": {"available", "missing"},
            "missing": set()
        }

        if value not in valid_transitions[self.__status]:
            raise ValueError(f"Invalid transition from {self.__status} to {value}")

        self.__status = value


# Demo
bc = BookCopy("C-2001")
bc.status = "on_loan"
bc.status = "available"
bc.status = "missing"
print(bc.status)


missing


### Exercise 17 — `Account` balance with non-negative invariant
Create `Account` with private `__balance` (float). Provide `deposit(amount)` and `withdraw(amount)` that update the balance via the `balance` property setter enforcing non-negative invariant and numeric coercion.

In [9]:
class Account:
    def __init__(self, opening_balance=0.0):
        self.__balance = 0.0
        self.balance = opening_balance

    @property
    def balance(self) -> float:
        return self.__balance

    @balance.setter
    def balance(self, value) -> None:
        try:
            value = float(value)
        except (TypeError, ValueError):
            raise ValueError(f"Balance must be a numeric value, got {value!r}")

        if value < 0:
            raise ValueError("Balance cannot be negative")

        self.__balance = value

    def deposit(self, amount) -> None:
        try:
            amount = float(amount)
        except (TypeError, ValueError):
            raise ValueError(f"Deposit amount must be numeric, got {amount!r}")

        if amount < 0:
            raise ValueError("Cannot deposit a negative amount")

        self.balance = self.balance + amount

    def withdraw(self, amount) -> None:
        try:
            amount = float(amount)
        except (TypeError, ValueError):
            raise ValueError(f"Withdrawal amount must be numeric, got {amount!r}")

        if amount < 0:
            raise ValueError("Cannot withdraw a negative amount")

        if amount > self.balance:
            raise ValueError("Insufficient funds")

        self.balance = self.balance - amount

# Demo
a = Account(5)
a.deposit("4.5")
print(a.balance)  # 9.5


9.5


### Exercise 18 — `HoldRequest` priority (1–5)
Create `HoldRequest` with private `__priority` (int). Property setter must coerce to int and clamp/validate to 1–5. Add read-only `created_by_member_id`.

In [10]:
class HoldRequest:
    def __init__(self, created_by_member_id: int, priority: int = 3):
        self.__created_by_member_id = int(created_by_member_id)
        self.__priority = None
        self.priority = priority

    @property
    def created_by_member_id(self) -> int:
        return self.__created_by_member_id

    @property
    def priority(self) -> int:
        return self.__priority

    @priority.setter
    def priority(self, value) -> None:
        try:
            value = int(value)
        except (TypeError, ValueError):
            raise ValueError(f"Priority must be an integer, got {value!r}")

        # Clamp priority to range [1, 5]
        if value < 1:
            value = 1
        elif value > 5:
            value = 5

        self.__priority = value

# Demo
hr = HoldRequest(42, 5)
print(hr.created_by_member_id, hr.priority)


42 5


### Exercise 19 — `CatalogRecord` with encapsulated fields & snapshot
Create `CatalogRecord` with private fields `__title`, `__author`, `__year` and respective properties (validate year 1450–2100). Add method `snapshot()` that returns a **new dict** with current public state (not the private attributes).

In [11]:
class CatalogRecord:
    def __init__(self, title: str, author: str, year):
        self.title = title
        self.author = author
        self.year = year

    @property
    def title(self) -> str:
        return self.__title

    @title.setter
    def title(self, value: str) -> None:
        if not isinstance(value, str) or not value.strip():
            raise ValueError("Title must be a non-empty string")
        self.__title = value.strip()

    @property
    def author(self) -> str:
        return self.__author

    @author.setter
    def author(self, value: str) -> None:
        if not isinstance(value, str) or not value.strip():
            raise ValueError("Author must be a non-empty string")
        self.__author = value.strip()

    @property
    def year(self) -> int:
        return self.__year

    @year.setter
    def year(self, value) -> None:
        try:
            value = int(value)
        except (TypeError, ValueError):
            raise ValueError(f"Year must be an integer, got {value!r}")

        if not (1450 <= value <= 2100):
            raise ValueError("Year must be between 1450 and 2100")

        self.__year = value

    def snapshot(self) -> dict:
        return {
            "title": self.title,
            "author": self.author,
            "year": self.year
        }

# Demo
rec = CatalogRecord("Design Patterns", "Gamma et al.", 1994)
print(rec.snapshot())


{'title': 'Design Patterns', 'author': 'Gamma et al.', 'year': 1994}


### Exercise 20 — `Settings` with encapsulated feature flags
Create `Settings` with private `__flags` dict (keys: 'allow_guest_checkout', 'enable_notifications'). Provide boolean properties for each flag that safely read/write the underlying dict while ensuring boolean values.

In [12]:
class Settings:
    def __init__(self, allow_guest_checkout: bool = False, enable_notifications: bool = True):
        self.__flags = {}
        self.allow_guest_checkout = allow_guest_checkout
        self.enable_notifications = enable_notifications

    @property
    def allow_guest_checkout(self) -> bool:
        return self.__flags.get("allow_guest_checkout", False)

    @allow_guest_checkout.setter
    def allow_guest_checkout(self, value) -> None:
        self.__flags["allow_guest_checkout"] = bool(value)

    @property
    def enable_notifications(self) -> bool:
        return self.__flags.get("enable_notifications", True)

    @enable_notifications.setter
    def enable_notifications(self, value) -> None:
        self.__flags["enable_notifications"] = bool(value)

# Demo
s = Settings()
s.allow_guest_checkout = 1
print(s.allow_guest_checkout, s.enable_notifications)


True True
